In [ ]:
import os
import sys
import pandas as pd
project_root = (os.path.dirname(os.path.abspath('.')))

# Add the project root to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.append(project_root)
    
import src.data.anonymizer as anonymizer
from src.data.id_mapper import IDMapper
from src.data.file_encoding import detect_file_encoding, batch_detect_encodings


Le paramètre `salt` dans la fonction `create_anonymous_id` joue un rôle crucial dans la génération d'ID anonymes de manière sécurisée et déterministe. Voici une explication détaillée de son rôle :

1. **Sécurité accrue** : Le `salt` est une valeur aléatoire ajoutée à l'identifiant (`identifier`) avant de générer le hachage. Cela empêche les attaques par dictionnaire et les attaques par force brute, car même si deux identifiants sont identiques, leurs hachages seront différents si des `salt` différents sont utilisés.

2. **Déterminisme** : Si un `salt` spécifique est fourni, la fonction générera toujours le même ID anonyme pour un identifiant donné. Cela est utile pour garantir que les mêmes identifiants d'origine produisent les mêmes ID anonymes à chaque exécution, tant que le même `salt` est utilisé.

3. **Génération aléatoire** : Si aucun `salt` n'est fourni, la fonction en génère un aléatoirement en utilisant `os.urandom(32)`. Cela garantit que chaque appel à la fonction avec le même identifiant produira un ID anonyme différent, ce qui peut être utile pour des besoins de sécurité spécifiques où le déterminisme n'est pas nécessaire.

Example d'usage:

In [ ]:
for i in range(3):
    hashed = anonymizer.create_anonymous_id(identifier='123123', salt=None)
    print(hashed)

# Whereas if we include a salt value, the hashed value will be different but consistent
print("\nWith salt:")
for i in range(3):
    hashed = anonymizer.create_anonymous_id(identifier='123123', salt='000')
    print(hashed)

### IDMapper class
It's easy to just create an unique identifer, and it will always return the same value once the salt is supplied (which it is).

In [ ]:
mapper = IDMapper()
[mapper.create_anonymous_id(original_id="123123") for i in range(3)]

# Build sample datasets

In [ ]:
path_raw = os.path.join(project_root, 'data/raw')

## MjÁlvarez

In [ ]:
# Print full paths of all files in the raw data directory
path = "".join([path_raw, '/MjÁlvarez'])
files = os.listdir(path)

# Read each file with pandas
df_list = []
for file in files:
    file_path = os.path.join(path, file)
    df = pd.read_csv(file_path)
    df_list.append(df)
    print(f"File: {os.path.basename(file_path)}")
    print(f"Shape: {df.shape}")

In [ ]:
# Given that the dataframes in df_list are adjacency matrices, suggest way to anonimize the rows and columns of the adjacency matrices

In [ ]:
df_list[0]

In [ ]:
len(df_list)

## TrustExperiment

In [ ]:
# Revise the encoding of the files TrustExperiment

encodings = batch_detect_encodings('../data/raw/TrustExperiment', pattern='*.csv')
for filename, encoding in encodings.items():
    print(f"{filename}: {encoding}")

In [ ]:
path = "".join([path_raw, '/TrustExperiment'])
files = os.listdir(path)


# Read files with the encoding in the encodings dictionary
df_list = []
for file in files:
    file_path = os.path.join(path, file)
    df = pd.read_csv(file_path, encoding=encodings[file])
    df_list.append(df)
    print(f"File: {os.path.basename(file_path)}")
    print(f"Shape: {df.shape}")

Now we've got to revise the current `MasterIDsFile.csv` and anonymize all the CSV for all the others.

In [ ]:
df_list[0]

In [ ]:
len(df_list[0].anonymousID.unique())

## Produce sample data 